In [1]:
# import the SimPy package 
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections

       
       
       
       
       
       .

In [2]:
def distribucion_llegada(env,puestos,cont):
    while True:
        cont["NT"] += 1
        r = random.random()
        #print("Distribucion de generacion de tipo de cliente: %5.2f"%r)
        if r <= cont["PE"]:
            ce = Customer(name = "clienteE%02d"%cont["NT"])
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            print("T=%5.2f %s: COLA EMERGENCIA - llegue con un IA: %5.2f"%(env.now,ce.name,ia))
            colas(puestos)
            env.process(ce.llamada_emergencia(env,puestos,cont))
        else: 
            cc = Customer(name = "clienteC%02d"%cont["NT"])
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            print("T=%5.2f %s: COLA COMERCIAL - llegue con un IA: %5.2f"%(env.now,cc.name,ia))
            colas(puestos)
            env.process(cc.llamada_comercial(env,puestos,cont))

In [3]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(self,env,puestos,cont):
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        emergencia, comercial = puestos[1].request(priority=0), puestos[0].request(priority=0)
        atendido = yield emergencia | comercial 
        #assert atendido == {emergencia}
        
        # Me atendieron por puestos comerciales?
        if comercial in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto comercial despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de emergencia"%(env.now,self.name))
            
        else:
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
            
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de emergencia"%(env.now,self.name))
            
            
    def llamada_comercial(self,env,puestos,cont):
        arrive = env.now
        with puestos[0].request(priority=1) as request:
            yield request
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA COMERCIAL - Me atendieron despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto comercial"%(env.now,self.name))
        

In [4]:
def colas(p):
    print ("Atendiendo:[c:%1d, e:%1d]"%(p[0].count,p[1].count))
    print ("En cola:[c:%1d, e:%1d]"%(len(p[0].queue),len(p[1].queue)))


In [5]:
def generar_TA(cont):
    #(a=5,b=615)
    a=5
    b=615
    ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
    
    while ta<a: 
        ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
        if ta>b:
            ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
            if ta>b:
                continue
            else:
                break
        
    while ta>b: 
        ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
        if ta<a:
            ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
            if ta<a:
                continue
            else:
                break
            
    return ta

Condiciones Iniciales

In [6]:
# Experimental data
NT = 0
maxTime = 14400 #segundos
#random.seed(RANDOM_SEED)
PE = 0.12 #Porcentaje Emergencia
PC = 0.88 #Porcentaje Comercial
NEC = 0 #Cantidad de gente que espero en cola menor a 40 segundos
PEC = 0.0 #Porcentaje de espera en cola menor a 40 segundos

# TA - distribucion normal
mu = 267.64 
sigma = 297.40

# IA - uniforme
IAMin = 1
IAMax = 60

# Contadores
ci = {"multitasking":0,
     "mu":mu,
     "sigma":sigma,
     "min":IAMin,
     "max":IAMax,
     "PE":PE,
     "PC":PC,
     "emer":0,
     "com":0,
     "NEC":NEC,
     "NT":NT}

In [15]:
# Puestos de atencion - variables de control
print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
ci["com"] = cantCom
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())
ci["emer"] = cantEmer

Ingrese cantidad de puestos comerciales
40
Ingrese cantidad de puestos de emergencia
5


Modelo de Simulacion

In [ ]:
env = simpy.Environment()
puestos = [simpy.PriorityResource(env,capacity=cantCom),
           simpy.PriorityResource(env,capacity=cantEmer)] # menor el numero priority, mayor es la prioridad
           
env.process(distribucion_llegada(env,puestos,ci))
env.run(until=maxTime)

T=15.74 clienteC927: COLA COMERCIAL - llegue con un IA: 15.74
Atendiendo:[c:0, e:0]
En cola:[c:0, e:0]
T=15.74, clienteC927: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:1, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC927: 336.35
T=26.16 clienteC928: COLA COMERCIAL - llegue con un IA: 10.42
Atendiendo:[c:1, e:0]
En cola:[c:0, e:0]
T=26.16, clienteC928: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:2, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC928: 437.63
T=49.75 clienteC929: COLA COMERCIAL - llegue con un IA: 23.59
Atendiendo:[c:2, e:0]
En cola:[c:0, e:0]
T=49.75, clienteC929: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:3, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC929: 138.00
T=108.12 clienteC930: COLA COMERCIAL - llegue con un IA: 58.36
Atendiendo:[c:3, e:0]
En cola:[c:0, e:0]
T=108.12, clienteC930: COLA COMERCIAL - Me atendieron despues de espera

Results

In [14]:
print("===========================================================================")
print("RESULTADOS:")
print("Clientes totales: %1d"%ci["NT"])
PEC = (ci["NEC"] / ci["NT"]) * 100
# Porcentaje de espera menor a 40 segundos
print("Porcentaje de espera en cola menor a 40 segundos: %5.2f"%PEC)
# Objetio: Mayor o igual al 90%

print('Cantidad de puestos comerciales',cantCom)
print('Cantidad de puestos de emergencia',cantEmer)

RESULTADOS:
Clientes totales: 926
Porcentaje de espera en cola menor a 40 segundos: 19.33
Cantidad de puestos comerciales 30
Cantidad de puestos de emergencia 5
